**laod features**

In [1]:
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix

#import models
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import NMF 
from sklearn.neighbors import KNeighborsClassifier

#Test:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

plt.style.use('ggplot')
# data processing libraries
import numpy as np
import pandas as pd

# plotting libraries
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')

# machine-learning libraries
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise
#from scipy.sparse import csr_matrix

# miscellaneous
import pickle

import numpy as np
from sklearn.decomposition import NMF 
import pandas as pd

# data processing libraries
import numpy as np
import pandas as pd

# plotting libraries
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')

# machine-learning libraries
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise
#from scipy.sparse import csr_matrix

# miscellaneous
import pickle

**load data**

In [2]:
movies = pd.read_csv('data/movies.csv')
rating = pd.read_csv('data/ratings.csv')
tags = pd.read_csv('data/tags.csv')
links = pd.read_csv('data/links.csv')

**create R matrix**

In [3]:
# calculate the number of ratings per movie
rating_count = rating.groupby('movieId')[['rating']].count()

# filter for movies with more than 20 ratings and extract the index
popular_movies = rating_count[rating_count['rating']>20].index

# filter the ratings matrix and only keep the popular movies
dfn = rating[rating['movieId'].isin(popular_movies)].copy()

# need to remake user ids and movie ids since they are not sequential
user_ids = dfn['userId'].unique()
user_id_map = {v:k for k,v in enumerate(user_ids)} #reduce with -1 user Id
dfn['userId'] = dfn['userId'].map(user_id_map)


movie_ids = dfn['movieId'].unique()
movie_id_map = {v:k for k,v in enumerate(movie_ids)} 
dfn['movieId'] = dfn['movieId'].map(movie_id_map)

# Initialize a sparse user-item rating matrix
# (data, (row_ind, col_ind) from scipy.sparse import csr_matrix
#careful csr fills NaN=0
R = csr_matrix((dfn['rating'], (dfn['userId'], dfn['movieId']))) 

#if all is well, the shape of the user-item matrix R will be (610, 1235)


#the movies.csv file should also the transformed accordingly

# filter out unpopular movies
movies = movies[movies['movieId'].isin(movie_ids)]
# redefine movie ids
movies['movieId'] = movies['movieId'].map(movie_id_map)

**show R matrix**

In [4]:
#get a column with all userIds
user_grouping = rating.groupby(['userId'])['rating'].mean()

In [5]:
R.todense()[0,:24]

matrix([[4., 4., 4., 5., 5., 3., 5., 4., 5., 5., 5., 3., 5., 4., 5., 3.,
         3., 5., 4., 4., 5., 4., 4., 5.]])

In [6]:
print(R)

  (0, 0)	4.0
  (0, 1)	4.0
  (0, 2)	4.0
  (0, 3)	5.0
  (0, 4)	5.0
  (0, 5)	3.0
  (0, 6)	5.0
  (0, 7)	4.0
  (0, 8)	5.0
  (0, 9)	5.0
  (0, 10)	5.0
  (0, 11)	3.0
  (0, 12)	5.0
  (0, 13)	4.0
  (0, 14)	5.0
  (0, 15)	3.0
  (0, 16)	3.0
  (0, 17)	5.0
  (0, 18)	4.0
  (0, 19)	4.0
  (0, 20)	5.0
  (0, 21)	4.0
  (0, 22)	4.0
  (0, 23)	5.0
  (0, 24)	4.0
  :	:
  (609, 1185)	3.0
  (609, 1186)	5.0
  (609, 1188)	5.0
  (609, 1189)	5.0
  (609, 1190)	3.0
  (609, 1192)	3.5
  (609, 1193)	4.0
  (609, 1194)	4.0
  (609, 1195)	4.0
  (609, 1196)	4.0
  (609, 1197)	3.0
  (609, 1199)	4.5
  (609, 1200)	5.0
  (609, 1203)	3.5
  (609, 1204)	3.0
  (609, 1211)	2.5
  (609, 1213)	3.5
  (609, 1214)	2.0
  (609, 1215)	4.0
  (609, 1216)	3.5
  (609, 1218)	4.0
  (609, 1219)	4.0
  (609, 1227)	4.0
  (609, 1231)	2.5
  (609, 1232)	3.0


In [7]:
#give titles to box axes and get a panda frame
Rt = pd.DataFrame(data=R.todense(), columns=movies['title'],index = user_grouping.index)
Rt.head(3)

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),GoldenEye (1995),"American President, The (1995)",Casino (1995),Sense and Sensibility (1995),...,Captain America: Civil War (2016),Doctor Strange (2016),The Martian (2015),Inside Out (2015),The Revenant (2015),"Big Short, The (2015)",Zootopia (2016),Arrival (2016),Rogue One: A Star Wars Story (2016),Logan (2017)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,4.0,4.0,5.0,5.0,3.0,5.0,4.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## A. Nearest Neighbors model 

**fit model**

In [8]:
modelnn = NearestNeighbors(metric='cosine')

# fit it to the Ratings matrix
modelnn.fit(Rt)

NearestNeighbors(metric='cosine')

In [9]:
def recommend_nn(query, model, Rt, k=10):
    """
    Filters and recommends the top k movies for any given input query based on a trained NMF model. 
    Returns a list of k movie ids.
    """
        
    # 1. construct new_user-item dataframe given the query
    new_user_dataframe =  pd.DataFrame(query, columns=movies['title'], index=['new_user'])
    # 1.2. fill the NaN
    new_user_dataframe_imputed = new_user_dataframe.fillna(0) #better mean
    # 2. scoring
    # calculates the distances to all other users in the data!
    similarity_scores, neighbor_ids = model.kneighbors(
    new_user_dataframe_imputed,
    n_neighbors=15,
    return_distance=True
    )

    # sklearn returns a list of predictions
    # extract the first and only value of the list

    neighbors_df = pd.DataFrame(
    data = {'neighbor_id': neighbor_ids[0], 'similarity_score': similarity_scores[0]}
    )
    
    # 3. ranking
    # only look at ratings for users that are similar!
    neighborhood = Rt.iloc[neighbor_ids[0]]
  
    
        # filter out movies already seen by the user
    neighborhood_filtered = neighborhood.drop(query.keys(),axis=1)
   

    # calculate the summed up rating for each movie
    # summing up introduces a bias for popular movies
    # averaging introduces bias for movies only seen by few users in the neighboorhood

    df_score = neighborhood_filtered.sum().sort_values(ascending=False)
    
    # return the top-k highest rated movie ids or titles
    df_score_ranked = df_score.sort_values(ascending=False).index.tolist()
    recommended = df_score_ranked[:k]
    return recommended, df_score

## B. NMF model

In [10]:
nmf_model = NMF(n_components=100, max_iter=1000, tol=0.001)

# fit it to the Ratings matrix
nmf_model.fit(Rt)

NMF(max_iter=1000, n_components=100, tol=0.001)

**create Q**

In [11]:
Q_matrix = nmf_model.components_
Q = pd.DataFrame(Q_matrix, columns=movies['title'], index=nmf_model.get_feature_names_out())
Q.head(2)

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),GoldenEye (1995),"American President, The (1995)",Casino (1995),Sense and Sensibility (1995),...,Captain America: Civil War (2016),Doctor Strange (2016),The Martian (2015),Inside Out (2015),The Revenant (2015),"Big Short, The (2015)",Zootopia (2016),Arrival (2016),Rogue One: A Star Wars Story (2016),Logan (2017)
nmf0,0.688077,0.000000,0.0,0.000000,0.0,0.0,0.0,4.535628,0.0,0.0,...,0.04119,0.038187,0.0,0.0,0.0,0.0,0.025754,0.0,0.0,0.0
nmf1,0.000000,0.101522,0.0,0.158818,0.0,0.0,0.0,0.119859,0.0,0.0,...,0.03087,0.026556,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [12]:
P_matrix = nmf_model.transform(Rt)

In [13]:
# to dataframe for better visualization
P = pd.DataFrame(
    data=P_matrix, 
    index=user_ids, 
    columns=nmf_model.get_feature_names_out()
)
P.head(3)

,nmf0,nmf1,nmf2,nmf3,nmf4,nmf5,nmf6,nmf7,nmf8,nmf9,...,nmf90,nmf91,nmf92,nmf93,nmf94,nmf95,nmf96,nmf97,nmf98,nmf99
1,0.000032,0.011989,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.020548,0.0,0.000000
2,0.000000,0.000000,0.0,0.035553,0.0,0.0,0.043818,0.001503,0.0,0.0,...,0.0,0.181075,0.000000,0.0,0.0,0.0,0.05806,0.096796,0.0,0.098998
3,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.004652,0.0,0.0,0.0,0.00000,0.000000,0.0,0.000000


In [14]:
R_hat_matrix = P.dot(Q)
R_hat_matrix.head(3)

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),GoldenEye (1995),"American President, The (1995)",Casino (1995),Sense and Sensibility (1995),...,Captain America: Civil War (2016),Doctor Strange (2016),The Martian (2015),Inside Out (2015),The Revenant (2015),"Big Short, The (2015)",Zootopia (2016),Arrival (2016),Rogue One: A Star Wars Story (2016),Logan (2017)
1,4.109607,3.838381,3.648823,5.184326,5.553539,2.828544,4.953443,3.709551,4.620017,5.002801,...,0.000371,0.001415,0.000000,0.000066,0.000000,0.000149,8.354772e-07,0.000000,0.005720,0.028869
2,0.311279,0.006899,0.066479,0.526822,0.849283,0.026666,0.018724,0.154521,0.006768,0.006642,...,0.000000,0.004053,0.009309,0.000000,0.122498,0.000000,1.392092e-01,0.014882,0.017493,0.002159
3,0.138353,0.056397,0.138907,0.096667,0.158814,0.080897,0.084234,0.116447,0.055982,0.065794,...,0.006193,0.011774,0.000203,0.074839,0.037392,0.045082,3.040123e-03,0.000316,0.047081,0.002513


In [15]:
R_hat = pd.DataFrame(
    data=R_hat_matrix, 
    index=user_grouping.index, 
    columns=movies['title']
)
R_hat.head(3)

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),GoldenEye (1995),"American President, The (1995)",Casino (1995),Sense and Sensibility (1995),...,Captain America: Civil War (2016),Doctor Strange (2016),The Martian (2015),Inside Out (2015),The Revenant (2015),"Big Short, The (2015)",Zootopia (2016),Arrival (2016),Rogue One: A Star Wars Story (2016),Logan (2017)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.109607,3.838381,3.648823,5.184326,5.553539,2.828544,4.953443,3.709551,4.620017,5.002801,...,0.000371,0.001415,0.000000,0.000066,0.000000,0.000149,8.354772e-07,0.000000,0.005720,0.028869
2,0.311279,0.006899,0.066479,0.526822,0.849283,0.026666,0.018724,0.154521,0.006768,0.006642,...,0.000000,0.004053,0.009309,0.000000,0.122498,0.000000,1.392092e-01,0.014882,0.017493,0.002159
3,0.138353,0.056397,0.138907,0.096667,0.158814,0.080897,0.084234,0.116447,0.055982,0.065794,...,0.006193,0.011774,0.000203,0.074839,0.037392,0.045082,3.040123e-03,0.000316,0.047081,0.002513


In [16]:
nmf_model.reconstruction_err_

551.7489217135868

In [17]:
def recommend_nmf(query, model, k=10):
    """
    Filters and recommends the top k movies for any given input query based on a trained NMF model. 
    Returns a list of k movie ids.
    """
    
    # 1. construct new_user-item dataframe given the query(votings of the new user)
   
    new_user_dataframe = pd.DataFrame(query, columns=movies['title'], index=['new_user'])
   
    new_user_dataframe_imputed =new_user_dataframe.fillna(0)

    P_new_user_matrix = model.transform(new_user_dataframe_imputed)
    # get as dataframe for a better visualizarion
    P_new_user = pd.DataFrame(P_new_user_matrix, 
                         columns = model.get_feature_names_out(),
                         index = ['new_user'])
    R_hat_new_user_matrix = np.dot(P_new_user,Q)
    # get as dataframe for a better visualizarion
    R_hat_new_user = pd.DataFrame(data=R_hat_new_user_matrix,
                         columns=movies['title'],
                         index = ['new_user'])
    R_hat_new_filtered = R_hat_new_user#.drop(new_user_query.keys(), axis=1)
    R_hat_new_filtered.T.sort_values(by=["new_user"], ascending=False).index.tolist()
    ranked =  R_hat_new_filtered.T.sort_values(by=["new_user"], ascending=False).index.tolist()
    # 2. scoring
    
        # calculate the score with the NMF model
    
    
    # 3. ranking
    
        # filter out movies already seen by the user
    
        # return the top-k highest rated movie ids or titles
    
    recommended = ranked[:k]
    return recommended, R_hat_new_filtered.T.sort_values(by=["new_user"], ascending=False)

## New user prediction

In [18]:
new_user_query = {"Toy Story (1995)": 5,
                 "Grumpier Old Men (1995)":2,
                 "Casino (1995)":3.5,
                 "Sabrina (1995)":4,
                 "GoldenEye (1995)":5}

In [19]:
new_user_dataframe = pd.DataFrame(new_user_query, columns=movies['title'], index=['new_user'])
new_user_dataframe

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),GoldenEye (1995),"American President, The (1995)",Casino (1995),Sense and Sensibility (1995),...,Captain America: Civil War (2016),Doctor Strange (2016),The Martian (2015),Inside Out (2015),The Revenant (2015),"Big Short, The (2015)",Zootopia (2016),Arrival (2016),Rogue One: A Star Wars Story (2016),Logan (2017)
new_user,5,NaN,2,NaN,NaN,4,5,NaN,3.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
recommend_nmf(new_user_query, nmf_model, k=10)

(['Toy Story (1995)',
  'Down Periscope (1996)',
  "Charlotte's Web (1973)",
  'Rumble in the Bronx (Hont faan kui) (1995)',
  'Die Hard: With a Vengeance (1995)',
  'Big Lebowski, The (1998)',
  'Taxi Driver (1976)',
  'Bridges of Madison County, The (1995)',
  'U.S. Marshals (1998)',
  'Ransom (1996)'],
                                             new_user
 title                                               
 Toy Story (1995)                            0.543301
 Down Periscope (1996)                       0.370611
 Charlotte's Web (1973)                      0.329411
 Rumble in the Bronx (Hont faan kui) (1995)  0.315683
 Die Hard: With a Vengeance (1995)           0.307988
 ...                                              ...
 WALL·E (2008)                               0.000000
 Bourne Identity, The (2002)                 0.000000
 Interstellar (2014)                         0.000000
 Dark Knight Rises, The (2012)               0.000000
 300 (2007)                                  

In [21]:
recommend_nn(new_user_query, modelnn, Rt)

(['Ransom (1996)',
  'Stand by Me (1986)',
  'Happy Gilmore (1996)',
  'Broken Arrow (1996)',
  'Down Periscope (1996)',
  'Beauty and the Beast (1991)',
  'City of Lost Children, The (Cité des enfants perdus, La) (1995)',
  'Die Hard: With a Vengeance (1995)',
  'Taxi Driver (1976)',
  'Heat (1995)'],
 title
 Ransom (1996)                                  33.0
 Stand by Me (1986)                             32.0
 Happy Gilmore (1996)                           31.0
 Broken Arrow (1996)                            27.0
 Down Periscope (1996)                          27.0
                                                ... 
 Big Daddy (1999)                                0.0
 Arachnophobia (1990)                            0.0
 South Park: Bigger, Longer and Uncut (1999)     0.0
 Wild Wild West (1999)                           0.0
 Logan (2017)                                    0.0
 Length: 1230, dtype: float64)

In [22]:
with open('./modelnn.pkl', 'wb') as file:
    pickle.dump(modelnn, file)

In [25]:
with open('./nmf_mode.pkl', 'wb') as file:
    pickle.dump(nmf_model, file)